In [1]:
import matplotlib.pyplot as plt
import numpy as np
# import networkx as nx
import os
import time
import json

In [16]:
path = "../outputs"
f = "path_21946_infos.json"

In [17]:
with open(os.path.join(path,f),"r") as infile:
    infos = json.load(infile)

In [18]:
infos

{'scale': 2,
 'pipeline': [[4, 80], [2, 50], [1, 0]],
 'data': [{'path_cells': [[542, 110],
    [544, 128],
    [544, 152],
    [544, 176],
    [528, 192],
    [512, 208],
    [504, 224],
    [496, 240],
    [496, 264],
    [496, 288],
    [496, 312],
    [496, 336],
    [480, 352],
    [480, 376],
    [464, 392],
    [448, 408],
    [448, 432],
    [448, 456],
    [448, 480],
    [432, 496],
    [432, 520],
    [432, 544],
    [432, 568],
    [432, 592],
    [432, 616],
    [416, 632],
    [408, 648],
    [408, 672],
    [400, 688],
    [400, 712],
    [400, 736],
    [384, 752],
    [368, 760],
    [352, 776],
    [352, 800],
    [352, 824],
    [352, 848],
    [352, 872],
    [352, 896],
    [352, 920],
    [352, 944],
    [336, 960],
    [320, 976],
    [320, 1000],
    [320, 1024],
    [320, 1048],
    [304, 1064],
    [288, 1080],
    [288, 1104],
    [272, 1120],
    [256, 1136],
    [240, 1152],
    [216, 1152],
    [200, 1168],
    [196, 1188]],
   'edgecosts': [[0.0, 0.0, 0.3